<a href="https://colab.research.google.com/github/tongtiger/KUanwjrrlrP/blob/master/kaggle/titanic/model_20_7_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
import pandas as pd
import numpy as np

In [7]:
dataset = pd.read_csv("/content/drive/Shared drives/Kaggle/titanic/data/train_data_20_7_28.csv")
dataset = pd.DataFrame(dataset)

dataset.drop(columns = ['Unnamed: 0', 'Ticket', 'Cabin', 'Fare'], inplace=True)

dataset

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,22.0,1,0,1
1,1,1,0,38.0,1,0,2
2,1,3,0,26.0,0,0,1
3,1,1,0,35.0,1,0,1
4,0,3,1,35.0,0,0,1
...,...,...,...,...,...,...,...
884,0,2,1,27.0,0,0,1
885,1,1,0,19.0,0,0,1
886,0,3,0,1.0,1,2,1
887,1,1,1,26.0,0,0,2


In [8]:
data = dataset.values

X = data[:, 1:6+1]
Y = data[:, 0]

In [9]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [40]:
from sklearn.model_selection import StratifiedKFold

n_fold = 40
skf = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = seed)

In [10]:
#X_train, Y_train, X_text, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = seed)

In [41]:
accuracy = []
z = 0
for train, test in skf.split(X, Y):
  model = Sequential()
  model.add(Dense(24, input_dim = 6, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  early_stopping_callback = EarlyStopping(monitor='val_loss', patience = 10)
  model.fit(X[train], Y[train], epochs = 100, batch_size=50, verbose=0, callbacks=[early_stopping_callback])
  print(model.evaluate(X[test], Y[test])[1])
  accuracy.append(model.evaluate(X[test], Y[test])[1])

print("검증 횟수 = ", len(accuracy))
print("모델 평균 정확도 = ", sum(accuracy)/len(accuracy))


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


23/23 [==============================] - 0s 724us/step
0.782608687877655
23/23 [==============================] - 0s 822us/step
0.8695651888847351
23/23 [==============================] - 0s 709us/step
0.739130437374115
23/23 [==============================] - 0s 782us/step
0.8695651888847351
23/23 [==============================] - 0s 728us/step
0.8695651888847351
23/23 [==============================] - 0s 806us/step
0.9130434989929199
23/23 [==============================] - 0s 867us/step
0.9130434989929199
23/23 [==============================] - 0s 750us/step
0.8260869383811951
23/23 [==============================] - 0s 752us/step
0.8260869383811951
22/22 [==============================] - 0s 875us/step
0.9545454382896423
22/22 [==============================] - 0s 995us/step
0.9090909361839294
22/22 [==============================] - 0s 790us/step
0.8181818127632141
22/22 [==============================] - 0s 767us/step
0.8636363744735718
22/22 [==============================] -

#K겹 교차 검증에 관한 내용

K겹 교차 검증에서, K개의 인스턴스가 생성되는데, 이때 생기는 인스턴스는 우리가 사용하는 모델이 '아니다'. 왜냐하면 모델은 인풋 데이터를 사용하여 결과값을 도출해내는 계산 과정을 이르는 말인데, K겹 교차 검증에서 각각의 인스턴스들은 인풋이 다르기 때문이다. 또한, K겹 교차 검증의 목적 자체가 더 나은 모델의 선택이 아닌, '한정된 데이터를 가지고 보다 타당한 모델 평가를 하기 위함'이다. 자세한 내용은 아래 링크를 참고하면 된다.

[K겹 교차 검증](https://stats.stackexchange.com/questions/52274/how-to-choose-a-predictive-model-after-k-fold-cross-validation)

In [42]:
model = Sequential()
model.add(Dense(24, input_dim = 6, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience = 10)
model.fit(X, Y, epochs = 100, batch_size=50, verbose=0, callbacks=[early_stopping_callback])

model.save("/content/drive/Shared drives/Kaggle/titanic/model/valacc_0.8152.model")

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [ ]:
'''import os

model_dir = "/content/drive/Shared drives/Kaggle/titanic/model/"
if not os.path.exists(model_dir):
  os.mkdir(model_dir)


modelpath = "/content/drive/Shared drives/Kaggle/titanic/model/{epoch:02d}_{val_loss:.4f}.hdf5"

checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', verbose = 1, save_best_only=True)

model.fit(X[train], Y[train], validation_split=0.2, epochs = 3500, batch_size=500, verbose=0, callbacks=[checkpointer])'''

In [45]:
from keras.models import load_model

predict_model = load_model("/content/drive/Shared drives/Kaggle/titanic/model/valacc_0.8152.model")

In [51]:
test_data = pd.read_csv("/content/drive/Shared drives/Kaggle/titanic/data/test_data_20_7_29.csv")
test_data = pd.DataFrame(test_data)
test_data.drop(columns=['Unnamed: 0'], inplace=True)
test_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Embarked
0,892.0,3.0,1,34.50000,0.0,0.0,1
1,893.0,3.0,0,47.00000,1.0,0.0,2
2,894.0,2.0,1,62.00000,0.0,0.0,1
3,895.0,3.0,1,27.00000,0.0,0.0,2
4,896.0,3.0,0,22.00000,1.0,1.0,2
...,...,...,...,...,...,...,...
413,1305.0,3.0,1,30.27259,0.0,0.0,2
414,1306.0,1.0,0,39.00000,0.0,0.0,3
415,1307.0,3.0,1,38.50000,0.0,0.0,2
416,1308.0,3.0,1,30.27259,0.0,0.0,2


In [58]:
#test_data = test_data.values
test_data

array([[8.920e+02, 3.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [8.930e+02, 3.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        2.000e+00],
       [8.940e+02, 2.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [1.307e+03, 3.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        2.000e+00],
       [1.308e+03, 3.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        2.000e+00],
       [1.309e+03, 3.000e+00, 1.000e+00, ..., 1.000e+00, 1.000e+00,
        3.000e+00]])

In [72]:
input = test_data[:, 1:6+1]

prediction = {"PassengerId": [], "Survived": []}

for answer in predict_model.predict(input): 
  if answer >= 0.5:
    answer = 1
  else:
    answer = 0
  prediction['Survived'].append(answer)

for data in test_data[:, 0]:
  prediction['PassengerId'].append(data)

result = pd.DataFrame(prediction)

result = result.astype('int')
print(result.info())

result.to_csv("/content/drive/Shared drives/Kaggle/titanic/data/result_20_7_29.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB
None
